In [3]:
import os
os.environ['SIRIUS_URL_CONSTS'] = 'http://127.0.0.1'

In [4]:
%gui qt5
%matplotlib qt5

from functools import partial as _partial
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
import matplotlib.gridspec as mpl_gs
import matplotlib.cm as cmap
from matplotlib import rcParams
rcParams.update(
    {'font.size': 16,'lines.linewidth': 2, 'axes.grid': True})

import pyaccel
import mathphys
from pymodels import si
from apsuite.commissioning_scripts.inj_traj_fitting import FitInjTraj

In [5]:
fit_traj = FitInjTraj()

In [6]:
np.random.seed(42)
x0, xl0, y0, yl0, de0 = -8.0e-3, 0.0e-3, 0.0e-3, 0.0, 0.01
twi = pyaccel.optics.Twiss(twiss=fit_traj.twiss[0], copy=True)
twi.betax = 100
trajx, trajy, trajsum = fit_traj.simulate_sofb(x0, xl0, y0, yl0, de0, twi=twi)
print(trajx.size)
vecs = fit_traj.do_fitting(trajx, trajy, tol=1e-8)

160


In [20]:
x, xl, y, yl, de = vecs[-1]
tmpl = '{:10s} ' + '{:^10.2f} '*5
ttmpl = '{:10s} ' + '{:^10s} '*5
print(ttmpl.format('', 'x [mm]', 'xl [mrad]', 'y [mm]', 'yl [mrad]', 'de [%]'))
print(tmpl.format('Real', x0*1e3, xl0*1e3, y0*1e3, yl0*1e3, de0*1e2))
print(tmpl.format('Fit', x*1e3, xl*1e3, y*1e3, yl*1e3, de*1e2))

             x [mm]   xl [mrad]    y [mm]   yl [mrad]    de [%]   
Real         -8.00       0.00       0.00       0.00       1.00    
Fit          -7.95      -0.01      -0.01       0.00       0.96    


In [21]:
f  = plt.figure(figsize=(9, 10))
gs = mpl_gs.GridSpec(3, 1)
gs.update(left=0.12, right=0.98, top=0.97, bottom=0.10, hspace=0.15)
ax = plt.subplot(gs[0, 0])
ay = plt.subplot(gs[1, 0])
asum = plt.subplot(gs[2, 0])

trajx_real, trajy_real = fit_traj.calc_traj(x0, xl0, y0, yl0, de0, size=trajx.size)
bpmpos = fit_traj.twiss.spos[fit_traj.bpm_idx]

ax.plot(bpmpos[:trajx_real.size], 1e3*trajx_real, '-x', label='Real')
ay.plot(bpmpos[:trajy_real.size], 1e3*trajy_real, '-x', label='Real')

ax.plot(bpmpos[:trajx.size], 1e3*trajx, '-d', label='SOFB')
ay.plot(bpmpos[:trajy.size], 1e3*trajy, '-d', label='SOFB')
asum.plot(bpmpos[:trajsum.size], trajsum)

trajx_fit, trajy_fit = fit_traj.calc_traj(*vecs[-1], size=trajx.size)
ax.plot(bpmpos[:trajx_fit.size], 1e3*trajx_fit, '-o', label='Fiting', linewidth=1)
ay.plot(bpmpos[:trajy_fit.size], 1e3*trajy_fit, '-o', label='Fiting', linewidth=1)

ay.legend()
asum.set_xlabel('Position [m]')
ax.set_ylabel('X [mm]')
ay.set_ylabel('Y [mm]')
asum.set_ylabel('Sum [counts]')
f.show()